# Text Ingestion and Cleaning

This notebook demonstrates how to load raw text data from the sandbox environment, perform basic normalization, and preview the results.

In [ ]:
import sys
import os
import pandas as pd

# Colab Setup
if 'google.colab' in sys.modules:
    !pip install -q pandas numpy
    if not os.path.exists('aire-researcher-sandbox'):
        !git clone https://github.com/YOUR_USERNAME/aire-researcher-sandbox.git
    %cd aire-researcher-sandbox

In [ ]:
# Load Data
data_path = 'data/sample_texts/articles_sample.csv'
if os.path.exists(data_path):
    df = pd.read_csv(data_path)
    print(f"Loaded {len(df)} articles.")
    display(df.head())
else:
    print("Data file not found. Please ensure synthetic data is generated.")

In [ ]:
# Basic Cleaning
def clean_text(text):
    if not isinstance(text, str): return ""
    return text.lower().strip()

df['clean_abstract'] = df['abstract'].apply(clean_text)
display(df[['abstract', 'clean_abstract']].head())